In [2]:
import os
from datetime import datetime
import pandas as pd
import re


cwd= os.getcwd()
path = os.path.join(cwd,'data')
fn=  os.path.join('dates.txt')

with open(fn, 'r') as f:
    content = f.read()

def extract_and_normalize_dates(text):
    date_patterns = [
        r'\d{1,2}/\d{1,2}/\d{2,4}',     
        r'\d{1,2}-\d{1,2}-\d{2,4}',     
        r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-. ]\d{1,2}[-, ]\d{2,4}',
        r'\d{1,2} (?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{2,4}',
        r'(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* \d{4}',
        r'\d{1,2}/\d{4}', 
        r'\d{4}' 
    ]

    date_regex = re.compile('|'.join(date_patterns))
    date_matches = date_regex.finditer(text)

    dates_with_index = []
    for match in date_matches:
        date_str = match.group()
        index = text.count('\n', 0, match.start())
        normalized_date = normalize_date(date_str)
        if normalized_date:
            dates_with_index.append((normalized_date, index))

    return sorted(dates_with_index, key=lambda x: x[0])

def normalize_date(date_str):
    current_year = datetime.now().year
    for fmt in ["%m/%d/%Y", "%m/%d/%y", "%m-%d-%Y", "%m-%d-%y", 
                "%b %d, %Y", "%b %d %Y", "%d %b %Y", "%b %Y", "%m/%Y", "%Y"]:
        try:
            normalized_date = datetime.strptime(date_str, fmt)
            # Check if the year is in the future
            if normalized_date.year > current_year:
                return None
            return normalized_date.strftime("%Y-%m-%d")
        except ValueError:
            continue

    if re.match(r'\d{1,2}/\d{4}', date_str):
        return datetime.strptime(date_str, "%m/%Y").strftime("%Y-%m-01")
    elif re.match(r'\d{4}', date_str):
        return datetime.strptime(date_str, "%Y").strftime("%Y-01-01")

    return None

text_content = content
sorted_dates_with_index = extract_and_normalize_dates(text_content)
df = pd.DataFrame(sorted_dates_with_index, columns=['res', 'index'])
df.reset_index(inplace=True)
df

,level_0,res,index
0,0,1971-04-10,9
1,1,1971-05-18,84
2,2,1971-07-08,2
3,3,1971-07-11,53
4,4,1971-09-12,28
...,...,...,...
428,428,2016-05-01,427
429,429,2016-05-30,141
430,430,2016-10-13,186
431,431,2016-10-19,161
